In [0]:
# imports 
from pyspark.ml.regression import DecisionTreeRegressor 
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator 
from mlflow.tracking import MlflowClient
import mlflow
import mlflow.spark 
import pandas as pd
from pyspark.ml.feature import RFormula

client = MlflowClient()

# load the datasets 
userhome = 'dbfs:/user/skim658@gwu.edu'

redDF = spark.read.parquet(userhome + '/final-project/reddf.parquet')
whiteDF = spark.read.parquet(userhome + '/final-project/whitedf.parquet')

In [0]:
# split the data set into train and test sets 
redTrainDF, redTestDF = redDF.repartition(8).randomSplit([0.8, 0.2], seed = 42)
print(redTrainDF.cache().count())
whiteTrainDF, whiteTestDF = whiteDF.repartition(8).randomSplit([0.8, 0.2], seed = 42)
print(whiteTrainDF.cache().count())

1276
3919

## Decision Tree with Cross Validation 

Max Tree Depth: 2, 5, 10; Max Bins: 10, 20, 40

## Red Wine

In [0]:
with mlflow.start_run(run_name = 'RED-DT-All-Features') as run:
  # model 
  dt = DecisionTreeRegressor(labelCol = 'quality')
  vecAssemblerDT = VectorAssembler(inputCols = [x for x in redTrainDF.columns if x != 'quality'], outputCol = 'features')
  # cross validation 
  dtparamGrid = ParamGridBuilder().addGrid(dt.maxDepth, [2, 5, 10]).addGrid(dt.maxBins, [10, 20, 40]).build()
  evaluator = RegressionEvaluator(predictionCol = 'prediction', labelCol = 'quality')
  cv = CrossValidator(estimator = dt, evaluator = evaluator, estimatorParamMaps = dtparamGrid, numFolds = 3, parallelism = 4)
  
  # pipeline 
  pipeline = Pipeline(stages = [vecAssemblerDT, cv])
  pipelineModel = pipeline.fit(redTrainDF)
  
  cvModel = pipelineModel.stages[-1]
  dtModel = cvModel.bestModel 
  
  testDF = vecAssemblerDT.transform(redTestDF)
  predDF = dtModel.transform(testDF)
  display(predDF.select('features', 'quality', 'prediction'))
  
  # log parameters 
  mlflow.log_param('features', 'all')
  mlflow.log_param('color', 'red')
  # log metric 
  rmse = evaluator.setMetricName('rmse').evaluate(predDF)
  mlflow.log_metric('rmse', rmse)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

features,quality,prediction
"Map(vectorType -> dense, length -> 11, values -> List(5.4, 0.74, 0.0, 1.2, 0.041, 16.0, 46.0, 0.99258, 4.01, 0.59, 12.5))",6,5.743589743589744
"Map(vectorType -> dense, length -> 11, values -> List(5.9, 0.61, 0.08, 2.1, 0.071, 16.0, 24.0, 0.99376, 3.56, 0.77, 11.1))",6,6.0
"Map(vectorType -> dense, length -> 11, values -> List(6.0, 0.5, 0.0, 1.4, 0.057, 15.0, 26.0, 0.99448, 3.36, 0.45, 9.5))",5,5.174887892376682
"Map(vectorType -> dense, length -> 11, values -> List(6.2, 0.64, 0.09, 2.5, 0.081, 15.0, 26.0, 0.99538, 3.57, 0.63, 12.0))",5,6.3076923076923075
"Map(vectorType -> dense, length -> 11, values -> List(6.4, 0.56, 0.15, 1.8, 0.078, 17.0, 65.0, 0.99294, 3.33, 0.6, 10.5))",6,5.703125
"Map(vectorType -> dense, length -> 11, values -> List(6.6, 0.815, 0.02, 2.7, 0.072, 17.0, 34.0, 0.9955, 3.58, 0.89, 12.3))",7,6.783132530120482
"Map(vectorType -> dense, length -> 11, values -> List(6.8, 0.91, 0.06, 2.0, 0.06, 4.0, 11.0, 0.99592, 3.53, 0.64, 10.9))",4,6.0
"Map(vectorType -> dense, length -> 11, values -> List(7.0, 0.36, 0.21, 2.4, 0.086, 24.0, 69.0, 0.99556, 3.4, 0.53, 10.1))",6,5.174887892376682
"Map(vectorType -> dense, length -> 11, values -> List(7.0, 0.685, 0.0, 1.9, 0.067, 40.0, 63.0, 0.9979, 3.6, 0.81, 9.9))",5,5.44
"Map(vectorType -> dense, length -> 11, values -> List(7.0, 0.78, 0.08, 2.0, 0.093, 10.0, 19.0, 0.9956, 3.4, 0.47, 10.0))",5,5.15625


In [0]:
display(dtModel)

treeNode
"{""index"":31,""featureType"":""continuous"",""prediction"":null,""threshold"":10.525,""categories"":null,""feature"":10,""overflow"":false}"
"{""index"":15,""featureType"":""continuous"",""prediction"":null,""threshold"":0.575,""categories"":null,""feature"":9,""overflow"":false}"
"{""index"":7,""featureType"":""continuous"",""prediction"":null,""threshold"":10.350000000000001,""categories"":null,""feature"":10,""overflow"":false}"
"{""index"":3,""featureType"":""continuous"",""prediction"":null,""threshold"":0.7525,""categories"":null,""feature"":1,""overflow"":false}"
"{""index"":1,""featureType"":""continuous"",""prediction"":null,""threshold"":0.525,""categories"":null,""feature"":2,""overflow"":false}"
"{""index"":0,""featureType"":null,""prediction"":5.174887892376682,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":2,""featureType"":null,""prediction"":4.8,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":5,""featureType"":""continuous"",""prediction"":null,""threshold"":0.065,""categories"":null,""feature"":2,""overflow"":false}"
"{""index"":4,""featureType"":null,""prediction"":4.5,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":6,""featureType"":null,""prediction"":5.15625,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"


In [0]:
# see the best model parameters 
# print(dtModel.extractParamMap())
paramDict = dtModel.extractParamMap()
tempDict = {}
for k, v in paramDict.items():
  tempDict[k.name] = v

bestMaxDepth = tempDict['maxDepth']
bestMaxBins = tempDict['maxBins']

tempDict = {'Max Depth': [bestMaxDepth], 'Max Bins': [bestMaxBins]}
bestCVModel = pd.DataFrame(tempDict, index = [0])
display(bestCVModel)

Max Depth,Max Bins
5,20


In [0]:
# get best model metrics 
experiment_id = run.info.experiment_id
runs = client.search_runs(experiment_id, order_by=["attributes.start_time desc"])
for run in runs:
  if run.data.params['mlModelClass'] == 'DecisionTreeRegressor' and int(run.data.params['maxBins']) == bestMaxBins and int(run.data.params['maxDepth']) == bestMaxDepth:
    print(run) 
    redMetricsDF = pd.DataFrame(run.data.metrics, index = [0])
    break
display(redMetricsDF)

<Run: data=<RunData: metrics={'avg_rmse': 0.6767975847554345, 'std_rmse': 0.013196260760210162}, params={'maxBins': '20',
 'maxDepth': '5',
 'mlEstimatorUid': 'DecisionTreeRegressor_203cb8c337f3',
 'mlModelClass': 'DecisionTreeRegressor'}, tags={'fit_uuid': '7aec74',
 'mlflow.parentRunId': '67f5aa83246342968e93f582fc66d5a9',
 'mlflow.rootRunId': '67f5aa83246342968e93f582fc66d5a9',
 'mlflow.user': 'prodbyskimmer@gmail.com',
 'runSource': 'mllibAutoTracking'}>, info=<RunInfo: artifact_uri='dbfs:/databricks/mlflow-tracking/929351302292913/d7a5cad116394c36b7fdc209e521ddd4/artifacts', end_time=None, experiment_id='929351302292913', lifecycle_stage='active', run_id='d7a5cad116394c36b7fdc209e521ddd4', run_uuid='d7a5cad116394c36b7fdc209e521ddd4', start_time=1606267080328, status='RUNNING', user_id=''>>

avg_rmse,std_rmse
0.6767975847554345,0.013196260760210162


In [0]:
# feature importance
pandasDF = pd.DataFrame(list(zip([x for x in redTrainDF.columns if x != 'quality'], dtModel.featureImportances)), columns=["feature", "importance"])
topFeatures = pandasDF.sort_values(["importance"], ascending=False)
topFeatures

Out[7]:

,feature,importance
10,alcohol,0.477595
9,sulphates,0.237685
1,volatile_acidity,0.099796
6,total_sulfur_dioxide,0.052981
5,free_sulfur_dioxide,0.034336
2,citric_acid,0.027481
0,fixed_acidity,0.026592
4,chlorides,0.021766
8,pH,0.011551
3,residual_sugar,0.010215


## White Wine

In [0]:
with mlflow.start_run(run_name = 'WHITE-DT-All-Features') as run:
  # model 
  dt = DecisionTreeRegressor(labelCol = 'quality')
  vecAssemblerDT = VectorAssembler(inputCols = [x for x in whiteTrainDF.columns if x != 'quality'], outputCol = 'features')
  # cross validation 
  dtparamGrid = ParamGridBuilder().addGrid(dt.maxDepth, [2, 5, 10]).addGrid(dt.maxBins, [10, 20, 40]).build()
  evaluator = RegressionEvaluator(predictionCol = 'prediction', labelCol = 'quality')
  cv = CrossValidator(estimator = dt, evaluator = evaluator, estimatorParamMaps = dtparamGrid, numFolds = 3, parallelism = 4)
  
  # pipeline 
  pipeline = Pipeline(stages = [vecAssemblerDT, cv])
  pipelineModel = pipeline.fit(whiteTrainDF)
  
  cvModel = pipelineModel.stages[-1]
  dtModel = cvModel.bestModel 
  
  testDF = vecAssemblerDT.transform(whiteTestDF)
  predDF = dtModel.transform(testDF)
  display(predDF.select('features', 'quality', 'prediction'))
  
  # log parameters 
  mlflow.log_param('features', 'all')
  mlflow.log_param('color', 'white')
  # log metric 
  rmse = evaluator.setMetricName('rmse').evaluate(predDF)
  mlflow.log_metric('rmse', rmse)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

features,quality,prediction
"Map(vectorType -> dense, length -> 11, values -> List(4.9, 0.47, 0.17, 1.9, 0.035, 60.0, 148.0, 0.98964, 3.27, 0.35, 11.5))",6,4.0
"Map(vectorType -> dense, length -> 11, values -> List(5.1, 0.35, 0.26, 6.8, 0.034, 36.0, 120.0, 0.99188, 3.38, 0.4, 11.5))",6,6.336309523809524
"Map(vectorType -> dense, length -> 11, values -> List(5.2, 0.3, 0.34, 1.5, 0.038, 18.0, 96.0, 0.98942, 3.56, 0.48, 13.0))",8,6.804347826086956
"Map(vectorType -> dense, length -> 11, values -> List(5.4, 0.29, 0.38, 1.2, 0.029, 31.0, 132.0, 0.98895, 3.28, 0.36, 12.4))",6,6.229166666666667
"Map(vectorType -> dense, length -> 11, values -> List(5.6, 0.18, 0.27, 1.7, 0.03, 31.0, 103.0, 0.98892, 3.35, 0.37, 12.9))",6,6.804347826086956
"Map(vectorType -> dense, length -> 11, values -> List(5.6, 0.26, 0.18, 1.4, 0.034, 18.0, 135.0, 0.99174, 3.32, 0.35, 10.2))",6,6.4
"Map(vectorType -> dense, length -> 11, values -> List(5.7, 0.2, 0.3, 2.5, 0.046, 38.0, 125.0, 0.99276, 3.34, 0.5, 9.9))",6,5.926881720430107
"Map(vectorType -> dense, length -> 11, values -> List(5.7, 0.28, 0.35, 1.2, 0.052, 39.0, 141.0, 0.99108, 3.44, 0.69, 11.3))",6,6.336309523809524
"Map(vectorType -> dense, length -> 11, values -> List(5.8, 0.22, 0.29, 1.3, 0.036, 25.0, 68.0, 0.98865, 3.24, 0.35, 12.6))",6,6.229166666666667
"Map(vectorType -> dense, length -> 11, values -> List(5.8, 0.23, 0.27, 1.8, 0.043, 24.0, 69.0, 0.9933, 3.38, 0.31, 9.4))",6,5.45


In [0]:
display(dtModel)

treeNode
"{""index"":31,""featureType"":""continuous"",""prediction"":null,""threshold"":10.850000000000001,""categories"":null,""feature"":10,""overflow"":false}"
"{""index"":15,""featureType"":""continuous"",""prediction"":null,""threshold"":0.2525,""categories"":null,""feature"":1,""overflow"":false}"
"{""index"":7,""featureType"":""continuous"",""prediction"":null,""threshold"":0.2075,""categories"":null,""feature"":1,""overflow"":false}"
"{""index"":3,""featureType"":""continuous"",""prediction"":null,""threshold"":0.9978100000000001,""categories"":null,""feature"":7,""overflow"":false}"
"{""index"":1,""featureType"":""continuous"",""prediction"":null,""threshold"":7.5,""categories"":null,""feature"":5,""overflow"":false}"
"{""index"":0,""featureType"":null,""prediction"":4.7,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":2,""featureType"":null,""prediction"":5.926881720430107,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":5,""featureType"":""continuous"",""prediction"":null,""threshold"":0.305,""categories"":null,""feature"":2,""overflow"":false}"
"{""index"":4,""featureType"":null,""prediction"":7.098039215686274,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":6,""featureType"":null,""prediction"":6.162162162162162,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"


In [0]:
# see the best model parameters 
paramDict = dtModel.extractParamMap()
tempDict = {}
for k, v in paramDict.items():
  tempDict[k.name] = v

bestMaxDepth = tempDict['maxDepth']
bestMaxBins = tempDict['maxBins']

tempDict = {'Max Depth': [bestMaxDepth], 'Max Bins': [bestMaxBins]}
bestCVModel = pd.DataFrame(tempDict, index = [0])
display(bestCVModel)

Max Depth,Max Bins
5,40


In [0]:
# get best model metrics 
experiment_id = run.info.experiment_id
runs = client.search_runs(experiment_id, order_by=["attributes.start_time desc"])
for run in runs:
  if run.data.params['mlModelClass'] == 'DecisionTreeRegressor' and int(run.data.params['maxBins']) == bestMaxBins and int(run.data.params['maxDepth']) == bestMaxDepth:
    print(run) 
    whiteMetricsDF = pd.DataFrame(run.data.metrics, index = [0])
    break
display(whiteMetricsDF)

<Run: data=<RunData: metrics={'avg_rmse': 0.7467741384386625, 'std_rmse': 0.011871040984318838}, params={'maxBins': '40',
 'maxDepth': '5',
 'mlEstimatorUid': 'DecisionTreeRegressor_6811c1488cc3',
 'mlModelClass': 'DecisionTreeRegressor'}, tags={'fit_uuid': '593398',
 'mlflow.parentRunId': 'def63076c86a44238bbff9a2467b28bb',
 'mlflow.rootRunId': 'def63076c86a44238bbff9a2467b28bb',
 'mlflow.user': 'prodbyskimmer@gmail.com',
 'runSource': 'mllibAutoTracking'}>, info=<RunInfo: artifact_uri='dbfs:/databricks/mlflow-tracking/929351302292913/9c02ce519ff8410f9018b6d303f978ed/artifacts', end_time=None, experiment_id='929351302292913', lifecycle_stage='active', run_id='9c02ce519ff8410f9018b6d303f978ed', run_uuid='9c02ce519ff8410f9018b6d303f978ed', start_time=1606267202384, status='RUNNING', user_id=''>>

avg_rmse,std_rmse
0.7467741384386625,0.011871040984318838


In [0]:
pandasDF = pd.DataFrame(list(zip([x for x in whiteTrainDF.columns if x != 'quality'], dtModel.featureImportances)), columns=["feature", "importance"])
topFeatures = pandasDF.sort_values(["importance"], ascending=False)
topFeatures

Out[12]:

,feature,importance
10,alcohol,0.529053
1,volatile_acidity,0.196905
5,free_sulfur_dioxide,0.126104
7,density,0.053797
3,residual_sugar,0.029487
2,citric_acid,0.021063
8,pH,0.014072
6,total_sulfur_dioxide,0.013836
0,fixed_acidity,0.010479
9,sulphates,0.005204


## Decision Tree with Hyperopt Hyperparameter Tuning

In [0]:
# imports
from hyperopt import hp 
from hyperopt import fmin, tpe, STATUS_OK, Trials

/databricks/python/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
 from imp import reload

## Red Wine

In [0]:
# define the search space 
search_space = {
  'maxDepth': hp.randint('maxDepth', 2, 30),
  'maxBins': hp.randint('maxBins', 10, 40)
}

def objective_function(params):
  with mlflow.start_run(nested = True):
    # hyperparameters to tune
    maxDepth = params['maxDepth']
    maxBins = params['maxBins']
    
    # model 
    vecAssemblerDT = VectorAssembler(inputCols = [x for x in redTrainDF.columns if x != 'quality'], outputCol = 'features')
    dt = DecisionTreeRegressor(labelCol = 'quality', maxBins = maxBins, maxDepth = maxDepth, seed = 42)
    
    # pipeline 
    pipeline = Pipeline(stages = [vecAssemblerDT, dt])
    pipelineModel = pipeline.fit(redTrainDF)
    
    # evaluate predictions 
    redPredDF = pipelineModel.transform(redTestDF)
    regressionEvaluator = RegressionEvaluator(predictionCol = 'prediction', labelCol = 'quality')
    rmse = regressionEvaluator.evaluate(redPredDF)
    
    # log parameters 
    mlflow.log_param('maxDepth', maxDepth)
    mlflow.log_param('maxBins', maxBins)
    mlflow.log_param('color', 'red')
    mlflow.log_param('features', 'all')
    mlflow.log_param('tuning', 'hyperopt')
    mlflow.log_metric('rmse', rmse)
  
  return {'loss': rmse, 'status': STATUS_OK}

# create parent run 
with mlflow.start_run(run_name = 'RED-DT-All-Features-Hyperopt') as run:
  num_evals = 30
  trials = Trials()
  best_hyperparam = fmin(fn = objective_function,
                        space = search_space,
                        algo = tpe.suggest,
                        max_evals = num_evals,
                        trials = trials)
  # log param and metric for the best model 
  for name, value in best_hyperparam.items():
    mlflow.log_param(name, value)
  mlflow.log_metric('loss', trials.best_trial['result']['loss'])
    

0%| | 0/30 [00:00<?, ?trial/s, best loss=?] 3%|▎ | 1/30 [00:06<02:59, 6.17s/trial, best loss: 0.779429672034433] 7%|▋ | 2/30 [00:13<03:05, 6.63s/trial, best loss: 0.779429672034433] 10%|█ | 3/30 [00:22<03:17, 7.31s/trial, best loss: 0.779429672034433] 13%|█▎ | 4/30 [00:32<03:32, 8.16s/trial, best loss: 0.779429672034433] 17%|█▋ | 5/30 [00:35<02:42, 6.51s/trial, best loss: 0.7189489880039093] 20%|██ | 6/30 [00:43<02:48, 7.03s/trial, best loss: 0.7189489880039093] 23%|██▎ | 7/30 [00:47<02:16, 5.95s/trial, best loss: 0.7170235088080664] 27%|██▋ | 8/30 [00:59<02:51, 7.79s/trial, best loss: 0.7170235088080664] 30%|███ | 9/30 [01:06<02:37, 7.49s/trial, best loss: 0.7170235088080664] 33%|███▎ | 10/30 [01:17<02:54, 8.71s/trial, best loss: 0.7170235088080664] 37%|███▋ | 11/30 [01:20<02:14, 7.08s/trial, best loss: 0.7170235088080664] 40%|████ | 12/30 [01:25<01:54, 6.34s/trial, best loss: 0.7170235088080664] 43%|████▎ | 13/30 [01:35<02:05, 7.37s/trial, best loss: 0.7170235088080664] 47%|████▋ | 14/30 [01:45<02:12, 8.27s/trial, best loss: 0.7170235088080664] 50%|█████ | 15/30 [01:54<02:08, 8.56s/trial, best loss: 0.7170235088080664] 53%|█████▎ | 16/30 [02:03<01:59, 8.55s/trial, best loss: 0.7170235088080664] 57%|█████▋ | 17/30 [02:08<01:37, 7.48s/trial, best loss: 0.7170235088080664] 60%|██████ | 18/30 [02:12<01:15, 6.32s/trial, best loss: 0.696403717075702]  63%|██████▎ | 19/30 [02:22<01:21, 7.43s/trial, best loss: 0.696403717075702] 67%|██████▋ | 20/30 [02:31<01:21, 8.16s/trial, best loss: 0.696403717075702] 70%|███████ | 21/30 [02:35<01:01, 6.83s/trial, best loss: 0.696403717075702] 73%|███████▎ | 22/30 [02:39<00:47, 5.89s/trial, best loss: 0.696403717075702] 77%|███████▋ | 23/30 [02:48<00:47, 6.82s/trial, best loss: 0.696403717075702] 80%|████████ | 24/30 [02:51<00:34, 5.82s/trial, best loss: 0.6838940275640455] 83%|████████▎ | 25/30 [02:55<00:25, 5.13s/trial, best loss: 0.6838940275640455] 87%|████████▋ | 26/30 [02:59<00:18, 4.71s/trial, best loss: 0.6838940275640455] 90%|█████████ | 27/30 [03:02<00:13, 4.40s/trial, best loss: 0.6838940275640455] 93%|█████████▎| 28/30 [03:09<00:10, 5.23s/trial, best loss: 0.6838940275640455] 97%|█████████▋| 29/30 [03:13<00:04, 4.81s/trial, best loss: 0.6838940275640455]100%|██████████| 30/30 [03:25<00:00, 6.75s/trial, best loss: 0.6838940275640455]100%|██████████| 30/30 [03:25<00:00, 6.83s/trial, best loss: 0.6838940275640455]

In [0]:
redResult = pd.DataFrame(best_hyperparam, index = [0])
display(redResult)

maxBins,maxDepth
10,4


In [0]:
bestMaxBins = best_hyperparam['maxBins']
bestMaxDepth = best_hyperparam['maxDepth']

In [0]:
# get best model metrics 
experiment_id = run.info.experiment_id
runs = client.search_runs(experiment_id, order_by=["attributes.start_time desc"])
for run in runs:
  if run.data.params['tuning'] == 'hyperopt' and int(run.data.params['maxBins']) == bestMaxBins and int(run.data.params['maxDepth']) == bestMaxDepth:
    print(run) 
    redMetricsDF = pd.DataFrame(run.data.metrics, index = [0])
    break
display(redMetricsDF)

<Run: data=<RunData: metrics={'rmse': 0.6838940275640455}, params={'color': 'red',
 'features': 'all',
 'maxBins': '10',
 'maxDepth': '4',
 'tuning': 'hyperopt'}, tags={'mlflow.databricks.cluster.id': '1124-235309-gins728',
 'mlflow.databricks.cluster.info': '{"cluster_id":"1124-235309-gins728","creator_user_id":2440320843892364,"driver":{"public_dns":"ec2-52-36-219-62.us-west-2.compute.amazonaws.com","ui_uri":"http://10.172.233.52:43661","memory_mb":10347,"node_id":"d8a92f58d19d4884a54dfebcc18afb58","node_aws_attributes":{"is_spot":false},"instance_id":"i-0aab5c13305b8bdc9","start_timestamp":1606261989912,"host_private_ip":"10.172.247.189","private_ip":"10.172.233.52"},"driver_uri":"https://10.172.233.52:6060","driver_ui_uri":"http://10.172.233.52:43661","spark_context_id":2197286493742322252,"driver_healthy":true,"jdbc_port":10000,"cluster_name":"final '
 'proj '
 '(clone)","spark_version":"7.4.x-cpu-ml-scala2.12","spark_conf":{"spark.databricks.delta.preview.enabled":"true"},"cluster_creator":"WEBAPP","aws_attributes":{"zone_id":"us-west-2c","first_on_demand":0,"availability":"ON_DEMAND","spot_bid_price_percent":100,"ebs_volume_count":0},"node_type_id":"dev-tier-node","driver_node_type_id":"dev-tier-node","autotermination_minutes":120,"enable_elastic_disk":false,"disk_spec":{"disk_count":0},"cluster_source":"UI","enable_jdbc_auto_start":true,"enable_jobs_autostart":true,"enable_local_disk_encryption":false,"package_label":"release__7.4.x-snapshot-cpu-ml-scala2.12__databricks-universe__head__ff14780__5233a8c__jenkins__2703f5a__format-2","state":"RUNNING","state_message":"","start_time":1606261989550,"terminated_time":0,"last_state_loss_time":0,"last_command_completion_time":1606262008900,"last_activity_time":1606262008900,"creator_org_id":7581726143880285,"num_workers":0,"cluster_memory_mb":15616,"cluster_cores":2.0,"creator_user_name":"prodbyskimmer@gmail.com","acl_path":"/clusters/1124-235309-gins728","init_scripts_safe_mode":false}',
 'mlflow.databricks.notebookID': '929351302292913',
 'mlflow.databricks.notebookPath': '/Users/prodbyskimmer@gmail.com/Final '
 'Research Project/04 - Decision Tree',
 'mlflow.databricks.notebookRevisionID': '1606267560389',
 'mlflow.databricks.webappURL': 'https://community.cloud.databricks.com',
 'mlflow.parentRunId': 'f2d3facf60c44294bb25670ccaff3d87',
 'mlflow.rootRunId': 'f2d3facf60c44294bb25670ccaff3d87',
 'mlflow.source.name': '/Users/prodbyskimmer@gmail.com/Final Research '
 'Project/04 - Decision Tree',
 'mlflow.source.type': 'NOTEBOOK',
 'mlflow.user': 'prodbyskimmer@gmail.com'}>, info=<RunInfo: artifact_uri='dbfs:/databricks/mlflow-tracking/929351302292913/cade79a52dcf4ca3bb23e0c605dafb9c/artifacts', end_time=1606267560300, experiment_id='929351302292913', lifecycle_stage='active', run_id='cade79a52dcf4ca3bb23e0c605dafb9c', run_uuid='cade79a52dcf4ca3bb23e0c605dafb9c', start_time=1606267556689, status='FINISHED', user_id=''>>

rmse
0.6838940275640455


## White Wine

In [0]:
def objective_function(params):
  with mlflow.start_run(nested = True):
    # hyperparameters to tune
    maxDepth = params['maxDepth']
    maxBins = params['maxBins']
    
    # model 
    vecAssemblerDT = VectorAssembler(inputCols = [x for x in whiteTrainDF.columns if x != 'quality'], outputCol = 'features')
    dt = DecisionTreeRegressor(labelCol = 'quality', maxBins = maxBins, maxDepth = maxDepth, seed = 42)
    
    # pipeline 
    pipeline = Pipeline(stages = [vecAssemblerDT, dt])
    pipelineModel = pipeline.fit(whiteTrainDF)
    
    # evaluate predictions 
    whitePredDF = pipelineModel.transform(whiteTestDF)
    regressionEvaluator = RegressionEvaluator(predictionCol = 'prediction', labelCol = 'quality')
    rmse = regressionEvaluator.evaluate(whitePredDF)
    
    # log parameters 
    mlflow.log_param('maxDepth', maxDepth)
    mlflow.log_param('maxBins', maxBins)
    mlflow.log_param('color', 'white')
    mlflow.log_param('features', 'all')
    mlflow.log_param('tuning', 'hyperopt')
    mlflow.log_metric('rmse', rmse)
  
  return {'loss': rmse, 'status': STATUS_OK}

# create parent run 
with mlflow.start_run(run_name = 'WHITE-DT-All-Features-Hyperopt') as run:
  num_evals = 30
  trials = Trials()
  best_hyperparam = fmin(fn = objective_function,
                        space = search_space,
                        algo = tpe.suggest,
                        max_evals = num_evals,
                        trials = trials)
  # log param and metric for the best model 
  for name, value in best_hyperparam.items():
    mlflow.log_param(name, value)
  mlflow.log_metric('loss', trials.best_trial['result']['loss'])

0%| | 0/30 [00:00<?, ?trial/s, best loss=?] 3%|▎ | 1/30 [00:04<02:14, 4.65s/trial, best loss: 0.7901732515923164] 7%|▋ | 2/30 [00:15<03:00, 6.45s/trial, best loss: 0.7901732515923164] 10%|█ | 3/30 [00:19<02:38, 5.86s/trial, best loss: 0.7766518306069717] 13%|█▎ | 4/30 [00:32<03:22, 7.80s/trial, best loss: 0.7766518306069717] 17%|█▋ | 5/30 [00:41<03:29, 8.37s/trial, best loss: 0.7766518306069717] 20%|██ | 6/30 [01:05<05:13, 13.05s/trial, best loss: 0.7766518306069717] 23%|██▎ | 7/30 [01:20<05:09, 13.47s/trial, best loss: 0.7766518306069717] 27%|██▋ | 8/30 [01:37<05:22, 14.66s/trial, best loss: 0.7766518306069717] 30%|███ | 9/30 [01:43<04:10, 11.91s/trial, best loss: 0.7766518306069717] 33%|███▎ | 10/30 [02:03<04:48, 14.43s/trial, best loss: 0.7766518306069717] 37%|███▋ | 11/30 [02:10<03:49, 12.07s/trial, best loss: 0.7766518306069717] 40%|████ | 12/30 [02:18<03:16, 10.90s/trial, best loss: 0.7766518306069717] 43%|████▎ | 13/30 [02:22<02:30, 8.83s/trial, best loss: 0.7766518306069717] 47%|████▋ | 14/30 [02:45<03:31, 13.23s/trial, best loss: 0.7766518306069717] 50%|█████ | 15/30 [02:49<02:37, 10.52s/trial, best loss: 0.7655037104961894] 53%|█████▎ | 16/30 [02:54<02:03, 8.80s/trial, best loss: 0.7617022714530096] 57%|█████▋ | 17/30 [03:11<02:24, 11.13s/trial, best loss: 0.7617022714530096] 60%|██████ | 18/30 [03:34<02:56, 14.67s/trial, best loss: 0.7617022714530096] 63%|██████▎ | 19/30 [03:54<03:01, 16.52s/trial, best loss: 0.7617022714530096] 67%|██████▋ | 20/30 [04:18<03:06, 18.63s/trial, best loss: 0.7617022714530096] 70%|███████ | 21/30 [04:22<02:08, 14.27s/trial, best loss: 0.7617022714530096] 73%|███████▎ | 22/30 [04:27<01:32, 11.54s/trial, best loss: 0.7617022714530096] 77%|███████▋ | 23/30 [04:33<01:07, 9.65s/trial, best loss: 0.7617022714530096] 80%|████████ | 24/30 [04:38<00:49, 8.30s/trial, best loss: 0.7617022714530096] 83%|████████▎ | 25/30 [04:43<00:36, 7.40s/trial, best loss: 0.7617022714530096] 87%|████████▋ | 26/30 [04:58<00:39, 9.80s/trial, best loss: 0.7617022714530096] 90%|█████████ | 27/30 [05:27<00:46, 15.42s/trial, best loss: 0.7617022714530096] 93%|█████████▎| 28/30 [05:53<00:36, 18.47s/trial, best loss: 0.7617022714530096] 97%|█████████▋| 29/30 [06:06<00:17, 17.05s/trial, best loss: 0.7617022714530096]100%|██████████| 30/30 [06:30<00:00, 19.18s/trial, best loss: 0.7617022714530096]100%|██████████| 30/30 [06:30<00:00, 13.03s/trial, best loss: 0.7617022714530096]

In [0]:
whiteResult = pd.DataFrame(best_hyperparam, index = [0])
display(whiteResult)

maxBins,maxDepth
12,7


In [0]:
bestMaxBins = best_hyperparam['maxBins']
bestMaxDepth = best_hyperparam['maxDepth']

In [0]:
# get best model metrics 
experiment_id = run.info.experiment_id
runs = client.search_runs(experiment_id, order_by=["attributes.start_time desc"])
for run in runs:
  if run.data.params['tuning'] == 'hyperopt' and int(run.data.params['maxBins']) == bestMaxBins and int(run.data.params['maxDepth']) == bestMaxDepth:
    print(run) 
    whiteMetricsDF = pd.DataFrame(run.data.metrics, index = [0])
    break
display(whiteMetricsDF)

<Run: data=<RunData: metrics={'rmse': 0.7617022714530096}, params={'color': 'white',
 'features': 'all',
 'maxBins': '12',
 'maxDepth': '7',
 'tuning': 'hyperopt'}, tags={'mlflow.databricks.cluster.id': '1124-235309-gins728',
 'mlflow.databricks.cluster.info': '{"cluster_id":"1124-235309-gins728","creator_user_id":2440320843892364,"driver":{"public_dns":"ec2-52-36-219-62.us-west-2.compute.amazonaws.com","ui_uri":"http://10.172.233.52:43661","memory_mb":10347,"node_id":"d8a92f58d19d4884a54dfebcc18afb58","node_aws_attributes":{"is_spot":false},"instance_id":"i-0aab5c13305b8bdc9","start_timestamp":1606261989912,"host_private_ip":"10.172.247.189","private_ip":"10.172.233.52"},"driver_uri":"https://10.172.233.52:6060","driver_ui_uri":"http://10.172.233.52:43661","spark_context_id":2197286493742322252,"driver_healthy":true,"jdbc_port":10000,"cluster_name":"final '
 'proj '
 '(clone)","spark_version":"7.4.x-cpu-ml-scala2.12","spark_conf":{"spark.databricks.delta.preview.enabled":"true"},"cluster_creator":"WEBAPP","aws_attributes":{"zone_id":"us-west-2c","first_on_demand":0,"availability":"ON_DEMAND","spot_bid_price_percent":100,"ebs_volume_count":0},"node_type_id":"dev-tier-node","driver_node_type_id":"dev-tier-node","autotermination_minutes":120,"enable_elastic_disk":false,"disk_spec":{"disk_count":0},"cluster_source":"UI","enable_jdbc_auto_start":true,"enable_jobs_autostart":true,"enable_local_disk_encryption":false,"package_label":"release__7.4.x-snapshot-cpu-ml-scala2.12__databricks-universe__head__ff14780__5233a8c__jenkins__2703f5a__format-2","state":"RUNNING","state_message":"","start_time":1606261989550,"terminated_time":0,"last_state_loss_time":0,"last_command_completion_time":1606262008900,"last_activity_time":1606262008900,"creator_org_id":7581726143880285,"num_workers":0,"cluster_memory_mb":15616,"cluster_cores":2.0,"creator_user_name":"prodbyskimmer@gmail.com","acl_path":"/clusters/1124-235309-gins728","init_scripts_safe_mode":false}',
 'mlflow.databricks.notebookID': '929351302292913',
 'mlflow.databricks.notebookPath': '/Users/prodbyskimmer@gmail.com/Final '
 'Research Project/04 - Decision Tree',
 'mlflow.databricks.notebookRevisionID': '1606267968937',
 'mlflow.databricks.webappURL': 'https://community.cloud.databricks.com',
 'mlflow.parentRunId': '67bd13231f494731846b336edb79b82c',
 'mlflow.rootRunId': '67bd13231f494731846b336edb79b82c',
 'mlflow.source.name': '/Users/prodbyskimmer@gmail.com/Final Research '
 'Project/04 - Decision Tree',
 'mlflow.source.type': 'NOTEBOOK',
 'mlflow.user': 'prodbyskimmer@gmail.com'}>, info=<RunInfo: artifact_uri='dbfs:/databricks/mlflow-tracking/929351302292913/1ad9d46e745449fdb5f63da66ce04a83/artifacts', end_time=1606267968843, experiment_id='929351302292913', lifecycle_stage='active', run_id='1ad9d46e745449fdb5f63da66ce04a83', run_uuid='1ad9d46e745449fdb5f63da66ce04a83', start_time=1606267963787, status='FINISHED', user_id=''>>

rmse
0.7617022714530096
